In [1]:
#%% Required Python Packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
np.random.seed(0)
import os

In [2]:
os.chdir('F:\SCR\Kaggle\PUGB')

In [3]:
train = pd.read_csv('train_V2.csv')
test = pd.read_csv('test_V2.csv')

## Data Preparation


In [4]:
# Check row with NaN value
train[train['winPlacePerc'].isnull()]

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
2744604,f70c74418bb064,12dfbede33f92b,224a123c53e008,0,0,0.0,0,0,0,1,...,0,0.0,0,0.0,0,0,0.0,0,0,NaN


In [5]:
# Drop row with NaN 'winPlacePerc' value
train.drop(2744604, inplace=True)

In [6]:
train = train.dropna()

In [8]:
train.columns

Index(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc'],
      dtype='object')

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4446965 entries, 0 to 4446965
Data columns (total 29 columns):
Id                 object
groupId            object
matchId            object
assists            int64
boosts             int64
damageDealt        float64
DBNOs              int64
headshotKills      int64
heals              int64
killPlace          int64
killPoints         int64
kills              int64
killStreaks        int64
longestKill        float64
matchDuration      int64
matchType          object
maxPlace           int64
numGroups          int64
rankPoints         int64
revives            int64
rideDistance       float64
roadKills          int64
swimDistance       float64
teamKills          int64
vehicleDestroys    int64
walkDistance       float64
weaponsAcquired    int64
winPoints          int64
winPlacePerc       float64
dtypes: float64(6), int64(19), object(4)
memory usage: 1017.8+ MB


In [40]:
train['damageDealt'] = train['damageDealt'].astype('int')
train['longestKill'] = train['longestKill'].astype('int')
train['rideDistance'] = train['rideDistance'].astype('int')
train['swimDistance'] = train['swimDistance'].astype('int')
train['walkDistance'] = train['walkDistance'].astype('int')
train['winPlacePerc'] = train['winPlacePerc'].astype('int')

In [27]:
#Indicamos las variables predictoras y debajo la variable objetivo. Cada uno con los nombres de las variables que tenéis en el fichero csv.
features_forest = train[['assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']]
## Indicamos el target
target = train.winPlacePerc

In [41]:
#Creamos la muestra de entrenamiento y de test, tanto para predictores como para la variable objetivo, siendo test el 40%
pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, target, test_size=.4)

In [24]:
#Comprobamos el tamaño de las diferentes muestras (pred=predictora; tar=target, objetivo). La salida en cada caso es una pareja de datos: el tamaño de la muestra y el número de variables
print(pred_train.shape)
print(pred_test.shape)
print(tar_train.shape)
print(tar_test.shape)

(2668179, 25)
(1778786, 25)
(2668179,)
(1778786,)


In [44]:
#Training The random forest
forest = RandomForestClassifier(n_jobs=2, random_state=0,n_estimators=100)
my_forest = forest.fit(features_forest, target)

In [45]:
print(my_forest.score(features_forest, target))

NameError: name 'my_forest' is not defined

In [56]:
# Train basic model
m1 = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features='sqrt',
                          n_jobs=-1)
m1.fit(X_train, y_train)
print_score(m1)

['mae train: ', 0.038876445239866289, 'mae val: ', 0.061615331583243754]


In [63]:
# Train final model
# You should get better results by increasing n_estimators
# and by playing around with the parameters
m3 = RandomForestRegressor(n_estimators=70, min_samples_leaf=3, max_features=0.5,
                          n_jobs=-1)
m3.fit(X_train, y_train)
print_score(m3)

['mae train: ', 0.03203275649680673, 'mae val: ', 0.057390563486910712]


## Kaggle submission

In [65]:
# Make submission ready for Kaggle
# We use our final Random Forest model (m3) to get the predictions
predictions = np.clip(a = m3.predict(test_pred), a_min = 0.0, a_max = 1.0)
pred_df = pd.DataFrame({'Id' : test['Id'], 'winPlacePerc' : predictions})

# Create submission file
pred_df.to_csv("submission.csv", index=False)

In [66]:
# Last check of submission
print('Head of submission: ')
display(pred_df.head())
print('Tail of submission: ')
display(pred_df.tail())

Head of submission: 


,Id,winPlacePerc
0,9329eb41e215eb,0.217230
1,639bd0dcd7bda8,0.887521
2,63d5c8ef8dfe91,0.697972
3,cf5b81422591d1,0.547064
4,ee6a295187ba21,0.893545


Tail of submission: 


,Id,winPlacePerc
1934169,a316c3a13887d5,0.770253
1934170,5312146b27d875,0.418772
1934171,fc8818b5b32ad3,0.893676
1934172,a0f91e35f8458f,0.809327
1934173,3696fc9f3a42b2,0.045944
